In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
import csv

In [2]:
# TODO: need to prompt the user to input email
# USER_EMAIL = "lbo@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"
USER_EMAIL = "stephanie.francor@gmail.com"
PASSWORD = "wDl37N16v$^a"
MAIN_PAGE_URL = "https://www.airbnb.com/login"
LISTING_PAGE_URL = "https://www.airbnb.com/hosting/listings"
RESERVATION_URL = "https://www.airbnb.com/hosting/reservations"

XPATH_email_btn = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/div/div/div/div[3]/div/div[4]/button/div/div[2]"
XPATH_login_form = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/div/div/div/form"
XPATH_listing_next_page_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div/div/div/div/section[1]/div/div/div[2]/div[3]/nav/div/button[2]"


In [3]:
driver = webdriver.Chrome() #Chromedriver
driver.get(MAIN_PAGE_URL)

In [5]:
def login(driver):
    # click continue with email with XPATH - doable
    email_btn = driver.find_element(By.XPATH, XPATH_email_btn)
    email_btn.click() 

    # try input user info - workable
    email_input = driver.find_element(By.NAME, "user[email]")
    email_input.send_keys(USER_EMAIL)

    # find "continue" button and click to submit email
    login_form = driver.find_element(By.XPATH, XPATH_login_form)
    login_form.submit()
    # TODO: after submitting, not shown (might need to sleep)
    time.sleep(2)

    # find password input form and input password
    pw_input = driver.find_element(By.NAME, "user[password]")
    pw_input.send_keys(PASSWORD)
    pw_input.submit()

    # TODO: SUCCESSFULLY LOGGED IN
    time.sleep(5)


In [6]:
login(driver)

In [7]:
# map script
driver.get(LISTING_PAGE_URL)

In [8]:
def is_suspended(status_info):
    status = status_info.split("</div>")[0]
    return status == "Suspended"

def append_one_entry(df, entry):
    # TODO: add status
    split_by_status = entry.split("<div class=\"_8hm2ou\"><div class=\"_l6x7wk\">")
    # TODO: add suspended
    if is_suspended(split_by_status[1]):
        return 
    after_listing_name = split_by_status[0].split("<div class=\"_t0tx82\">")[1] # after listing name
    before_modified = after_listing_name.split("<div class=\"_1vy3vsk\">")[0] # before modified
    infos = before_modified.split("</div>")

    if (len(infos) != 3): # if internal title is absent, length == 2
        return
    
    listing_name = infos[0][22:]
    internal_title = infos[1].split(">")[1]
    df["listing name"].append(listing_name)
    df["internal title"].append(internal_title)

def append_all_entries(df, html_source):
    entries = html_source.split("<tr data-testid=\"host-listings-row-")[1:]
    entries[-1] = entries[-1].split("<td class=\"_10v9brh\"></td></tr>")[0]
    for entry in entries:
        append_one_entry(df, entry)


def append_all_pages(driver, df):
    html_source = ""
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries(df, html_source)
        next_page_btn = driver.find_element(By.XPATH, XPATH_listing_next_page_btn)
        next_page_btn.click()
        time.sleep(1)


In [9]:
df_dict = {
    "listing name": [],
    "internal title": []
}
append_all_pages(driver, df_dict)

In [10]:
df = pd.DataFrame(df_dict)
df

,listing name,internal title
0,110 Acres! PrivateLake* Scenic* Fireplace*Gril...,Kentucky - Pinecone Lodg
1,Lit Deck 3BR2BA Artistic Hideaway Marietta Sq bll,738 Bellemeade Way - Atlanta
2,*King Bed*Pets* 1-Story Home Covered Patio wst,3175 Westwood - Atlanta
3,Sleeps5*Luxury*HighFloor*Balcony *Renovated 15I,300 Peachtree 15I - Atlanta - Uhome
4,Romantic Cabin w/ in-room Jacuzzi* WIFI* Firep...,194 Alpine - Elsa
...,...,...
188,"Newly Renovated unit A in 4-plex,8min frm dwntwn",unit A - Indianapolis
189,Newly Renovated Unit B 8 mins frm downtown,Newly Renovated Unit B - Indianapolis
190,"Newly Renovated Unit D, 8min frm dwntown",Newly Renovated Unit D - Indianapolis
191,Discard: bvu,1086 Briar Vista Upper - Atlanta


In [11]:
df.to_csv("./listing_internal_mapping.csv", index=False)

In [116]:
# TODO: to close the driver, uncommnet the following line and run
# driver.close()